In [2]:
import os
import tarfile
import boto3
from urllib.parse import urlparse

import tensorflow as tf
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

# =========================
# Config
# =========================
H5_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-19-03-37-10-946/output/model.tar.gz"
ENDPOINT_NAME = "BMI-predcitor-v3"
INSTANCE_TYPE = "ml.g4dn.xlarge"
FRAMEWORK_VERSION = "2.11.0"

SAVEDMODEL_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-19-03-37-10-946/output/savedmodel/model.tar.gz"

# Existing feature extraction endpoint
FEATURE_ENDPOINT_NAME = "feature-extraction-efficientnetb7"

# Scalers
SCALER_TARGETS_S3_URI = "s3://ai-bmi-predictor/scalers/scaler_targets.pkl"
SCALER_ROBUST_FEATURES_S3_URI = "s3://ai-bmi-predictor/scalers/scaler_robust_features.pkl"
SCALER_HEIGHT_S3_URI = "s3://ai-bmi-predictor/scalers/scaler_standard_features.pkl"

# =========================
# Role inference
# =========================
try:
    ROLE  # type: ignore
except NameError:
    try:
        from sagemaker import get_execution_role
        ROLE = get_execution_role()
    except Exception:
        ROLE = None

if not ROLE:
    raise ValueError("ROLE is None. Set ROLE to your SageMaker execution role ARN if running outside SageMaker.")

# =========================
# Helpers
# =========================
def parse_s3_uri(uri: str):
    p = urlparse(uri)
    if p.scheme != "s3" or not p.netloc or not p.path:
        raise ValueError(f"Invalid S3 URI: {uri}")
    return p.netloc, p.path.lstrip("/")

def safe_extract(tar: tarfile.TarFile, path: str):
    abs_path = os.path.abspath(path)
    for member in tar.getmembers():
        member_path = os.path.abspath(os.path.join(path, member.name))
        if not member_path.startswith(abs_path + os.sep) and member_path != abs_path:
            raise Exception(f"Blocked path traversal attempt: {member.name}")
    tar.extractall(path)

def upload_file_to_s3(local_path: str, s3_uri: str):
    bucket, key = parse_s3_uri(s3_uri)
    boto3.client("s3").upload_file(local_path, bucket, key)
    return f"s3://{bucket}/{key}"

# =========================
# 0) Write inference code BEFORE deploy
# =========================
source_dir = "bmi_inference_code"
os.makedirs(source_dir, exist_ok=True)

# requirements (installed in container)
with open(os.path.join(source_dir, "requirements.txt"), "w", encoding="utf-8") as f:
    f.write("numpy\nscikit-learn\nrequests\n")

# inference.py (preprocess inside endpoint)
inference_py = r'''
import os
import json
import base64
import pickle

import numpy as np
import boto3
import requests

_s3 = boto3.client("s3")
_runtime = boto3.client("sagemaker-runtime")

FEATURE_ENDPOINT_NAME = os.environ.get("FEATURE_ENDPOINT_NAME", "feature-extraction-efficientnetb7")
SCALER_TARGETS_S3_URI = os.environ["SCALER_TARGETS_S3_URI"]
SCALER_ROBUST_FEATURES_S3_URI = os.environ["SCALER_ROBUST_FEATURES_S3_URI"]
SCALER_HEIGHT_S3_URI = os.environ["SCALER_HEIGHT_S3_URI"]

TARGET_COLS = [
    "ankle", "arm-length", "bicep", "calf", "chest", "forearm", "hip",
    "leg-length", "shoulder-breadth", "shoulder-to-crotch", "thigh",
    "waist", "wrist", "weight_kg"
]

def _parse_s3_uri(uri: str):
    if not uri.startswith("s3://"):
        raise ValueError(f"Invalid S3 URI: {uri}")
    x = uri[5:]
    bucket, key = x.split("/", 1)
    return bucket, key

def _load_pickle_from_s3(s3_uri: str):
    b, k = _parse_s3_uri(s3_uri)
    obj = _s3.get_object(Bucket=b, Key=k)
    return pickle.loads(obj["Body"].read())

# Load scalers once (cold start)
TARGET_SCALER = _load_pickle_from_s3(SCALER_TARGETS_S3_URI)
ROBUST_FEATURES_SCALER = _load_pickle_from_s3(SCALER_ROBUST_FEATURES_S3_URI)
HEIGHT_SCALER = _load_pickle_from_s3(SCALER_HEIGHT_S3_URI)

def _gender_to_code(g):
    # IMPORTANT: adjust if your training mapping differs
    s = str(g).strip().lower()
    if s in ["female", "f", "0"]:
        return 0
    if s in ["male", "m", "1"]:
        return 1
    raise ValueError(f"Unsupported gender: {g}")

def _get_image_bytes(payload: dict, key_b64: str, key_s3: str):
    # Accept either base64 image or s3 uri
    if payload.get(key_b64):
        return base64.b64decode(payload[key_b64])
    if payload.get(key_s3):
        b, k = _parse_s3_uri(payload[key_s3])
        obj = _s3.get_object(Bucket=b, Key=k)
        return obj["Body"].read()
    raise ValueError(f"Provide '{key_b64}' or '{key_s3}'")

def _invoke_feature_endpoint(image_bytes: bytes):
    # Send image as base64 JSON
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    req = json.dumps({"instances": [{"b64": b64}]})

    resp = _runtime.invoke_endpoint(
        EndpointName=FEATURE_ENDPOINT_NAME,
        ContentType="application/json",
        Accept="application/json",
        Body=req.encode("utf-8"),
    )
    body = resp["Body"].read().decode("utf-8")
    data = json.loads(body)

    # Common response shapes
    if "predictions" in data:
        return np.asarray(data["predictions"][0], dtype=np.float32).reshape(-1)
    if "features" in data:
        return np.asarray(data["features"], dtype=np.float32).reshape(-1)
    if isinstance(data, list):
        return np.asarray(data, dtype=np.float32).reshape(-1)

    raise ValueError(f"Unrecognized feature endpoint response: {data}")

def handler(data, context):
    # 1) Read JSON
    raw = data.read()
    payload = json.loads(raw.decode("utf-8")) if raw else {}

    height_cm = float(payload["height_cm"])
    gender_code = _gender_to_code(payload["gender"])

    front_bytes = _get_image_bytes(payload, "front_image_b64", "front_image_s3")
    side_bytes  = _get_image_bytes(payload, "side_image_b64",  "side_image_s3")

    # 2) Feature extraction (front + side)
    front_feat = _invoke_feature_endpoint(front_bytes)
    side_feat  = _invoke_feature_endpoint(side_bytes)

    # 3) Final feature order: front_feats, side_feats, gender, height
    cnn = np.concatenate([front_feat, side_feat], axis=0).astype(np.float32).reshape(1, -1)

    cnn_scaled = ROBUST_FEATURES_SCALER.transform(cnn).astype(np.float32)
    height_scaled = HEIGHT_SCALER.transform(np.array([[height_cm]], dtype=np.float32)).astype(np.float32)

    final_x = np.concatenate(
        [cnn_scaled, np.array([[gender_code]], dtype=np.float32), height_scaled],
        axis=1
    )

    # 4) Call the local TF Serving model
    tfs_req = json.dumps({"instances": final_x.tolist()})
    tfs_resp = requests.post(context.rest_uri, data=tfs_req)

    if tfs_resp.status_code != 200:
        raise ValueError(tfs_resp.content.decode("utf-8"))

    pred = tfs_resp.json()
    y_scaled = np.asarray(pred["predictions"], dtype=np.float32)

    # 5) Inverse-scale targets
    y = TARGET_SCALER.inverse_transform(y_scaled)[0].tolist()
    out = {TARGET_COLS[i]: float(y[i]) for i in range(len(TARGET_COLS))}

    return json.dumps(out).encode("utf-8"), "application/json"
'''
with open(os.path.join(source_dir, "inference.py"), "w", encoding="utf-8") as f:
    f.write(inference_py)

print("✅ Wrote bmi_inference_code/inference.py and requirements.txt")

# =========================
# 1) Download & extract H5 from tar.gz
# =========================
workdir = "bmi_model_work"
os.makedirs(workdir, exist_ok=True)

local_in_tar = os.path.join(workdir, "model.tar.gz")
extract_dir = os.path.join(workdir, "extracted")
os.makedirs(extract_dir, exist_ok=True)

in_bucket, in_key = parse_s3_uri(H5_TAR_S3_URI)
boto3.client("s3").download_file(in_bucket, in_key, local_in_tar)
print(f"✅ Downloaded: {H5_TAR_S3_URI}")

with tarfile.open(local_in_tar, "r:gz") as tar:
    safe_extract(tar, extract_dir)

# Find .h5
h5_path = None
for root, _, files in os.walk(extract_dir):
    for fn in files:
        if fn.lower().endswith((".h5", ".hdf5")):
            h5_path = os.path.join(root, fn)
            break
    if h5_path:
        break

if not h5_path:
    raise ValueError("❌ No .h5 found inside the downloaded archive.")

print(f"✅ Found H5: {h5_path}")

# =========================
# 2) Convert H5 -> SavedModel (model/1/)
# =========================
model = tf.keras.models.load_model(h5_path, compile=False)
print("✅ Loaded Keras model")

serving_root = os.path.join(workdir, "model")
version_dir = os.path.join(serving_root, "1")
os.makedirs(version_dir, exist_ok=True)

tf.saved_model.save(model, version_dir)
print(f"✅ Exported SavedModel to: {version_dir}")

# =========================
# 3) Pack SavedModel tar.gz (contains model/1/...)
# =========================
local_out_tar = os.path.join(workdir, "savedmodel.tar.gz")
with tarfile.open(local_out_tar, "w:gz") as tar:
    tar.add(serving_root, arcname="model")

print(f"✅ Repacked SavedModel tar: {local_out_tar}")

# =========================
# 4) Upload SavedModel tarball
# =========================
uploaded_s3 = upload_file_to_s3(local_out_tar, SAVEDMODEL_TAR_S3_URI)
print(f"✅ Uploaded SavedModel artifact to: {uploaded_s3}")

# =========================
# 5) Deploy endpoint (preprocess happens inside inference.py)
# =========================
session = sagemaker.Session()

tf_model = TensorFlowModel(
    model_data=uploaded_s3,
    role=ROLE,
    framework_version=FRAMEWORK_VERSION,
    sagemaker_session=session,
    entry_point="inference.py",
    source_dir=source_dir,
    env={
        "FEATURE_ENDPOINT_NAME": FEATURE_ENDPOINT_NAME,
        "SCALER_TARGETS_S3_URI": SCALER_TARGETS_S3_URI,
        "SCALER_ROBUST_FEATURES_S3_URI": SCALER_ROBUST_FEATURES_S3_URI,
        "SCALER_HEIGHT_S3_URI": SCALER_HEIGHT_S3_URI,
    },
)

predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
)

print(f"\n✅ Deployed endpoint: {ENDPOINT_NAME}")
print(f"   • Artifact: {uploaded_s3}")


✅ Wrote bmi_inference_code/inference.py and requirements.txt
✅ Downloaded: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-19-03-37-10-946/output/model.tar.gz
✅ Found H5: bmi_model_work/extracted/eff_ann_version8.h5
✅ Loaded Keras model
INFO:tensorflow:Assets written to: bmi_model_work/model/1/assets


INFO:tensorflow:Assets written to: bmi_model_work/model/1/assets


✅ Exported SavedModel to: bmi_model_work/model/1
✅ Repacked SavedModel tar: bmi_model_work/savedmodel.tar.gz
✅ Uploaded SavedModel artifact to: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-19-03-37-10-946/output/savedmodel/model.tar.gz
--------!
✅ Deployed endpoint: BMI-predcitor-v3
   • Artifact: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-19-03-37-10-946/output/savedmodel/model.tar.gz
